# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Load the csv exported in Part I
output_data_file = pd.read_csv("../WeatherPy/output_data/cities.csv")
output_data_file.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Port Lincoln,53,AU,1607362995,67,-34.73,135.87,54.37,8.08
1,Barrow,90,US,1607362936,77,71.29,-156.79,1.40,8.05
2,Jacareacanga,90,BR,1607363469,94,-6.22,-57.75,77.00,5.82
3,Mataura,37,NZ,1607362931,78,-46.19,168.86,48.02,2.44
4,Punta Arenas,75,CL,1607362819,55,-53.15,-70.92,62.60,16.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [12]:
m = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(output_data_file[['Latitude','Longitude']], weights=output_data_file['Humidity'],max_intensity=8000, point_radius=10.0
)
#point markers layer:
m.add_layer(heatmap_layer)
m

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
output_data_file

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Port Lincoln,53,AU,1607362995,67,-34.73,135.87,54.37,8.08
1,Barrow,90,US,1607362936,77,71.29,-156.79,1.40,8.05
2,Jacareacanga,90,BR,1607363469,94,-6.22,-57.75,77.00,5.82
3,Mataura,37,NZ,1607362931,78,-46.19,168.86,48.02,2.44
4,Punta Arenas,75,CL,1607362819,55,-53.15,-70.92,62.60,16.11
...,...,...,...,...,...,...,...,...,...
581,Gizo Government Station,48,SB,1607363558,73,-8.10,156.84,82.58,3.18
582,Stornoway,40,GB,1607363558,86,58.21,-6.39,42.01,8.05
583,Xianshuigu,0,CN,1607363402,37,38.98,117.38,21.20,6.71
584,Morehead,75,US,1607363559,51,37.27,-87.18,41.00,6.93


In [18]:
output_data_file.loc[(output_data_file['Max Temp'] <= 30) & (output_data_file['Humidity'] <= 60)]

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
203,Krasnoarmeysk,24,RU,1607363496,43,51.02,45.70,21.99,4.00
386,Asău,100,RO,1607363533,22,46.43,26.40,27.00,4.23
583,Xianshuigu,0,CN,1607363402,37,38.98,117.38,21.20,6.71


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = output_data_file.loc[(output_data_file['Max Temp'] <= 30) & (output_data_file['Humidity'] <= 60)].copy()

In [24]:
hotel_df['Hotel'] = ""

In [25]:
# geocoordinates
target_coordinates = "51.02, 45.70"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [29]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 51.0163795,
                    "lng": 45.6983185
                },
                "viewport": {
                    "northeast": {
                        "lat": 51.0178117302915,
                        "lng": 45.6998457802915
                    },
                    "southwest": {
                        "lat": 51.01511376970851,
                        "lng": 45.6971478197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Gostinitsa Komfort",
            "photos": [
                {
                    "height": 3264,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/116266201282455007268\">\u041d\u0438\u

In [30]:
print(places_data["results"][0]["name"])

Gostinitsa Komfort


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
